In [1]:
from codex import Codex

import logging
import os
import json
from importlib import resources

from codex.database.espresso import run_helpdoc, generate_database, _extract_vars

from pymatgen.io.vasp.inputs import Incar, Poscar, Potcar, Kpoints

from tabulate import tabulate

from lxml.html.clean import Cleaner
from lxml import etree

logging.basicConfig(level=logging.INFO)

# VASP database exploration

In [2]:
incar = Incar.from_file("example/INCAR_Sr3PbO")
incar = Incar.from_file("/Users/ashour/Google Drive/My Drive/Research/Griffin/Fe3O4/INCAR")

In [ ]:
incar_string = incar.get_string(sort_keys=False)

In [4]:
incar_filename = "/Users/ashour/Google Drive/My Drive/Research/Griffin/Fe3O4/INCAR"
codex = Codex(incar_filename, "vasp", "1686736265")

In [2]:
codex = Codex("example/sr3pbo_bands.in", "qe", "7.2")

KeyError: 'summary'

In [ ]:
code = "vasp"
dbversion = "1686736265"
base_db_dir = resources.files("codex.database")
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)
database = {"vasp": database} if code == "vasp" else database

In [ ]:
package = "vasp"
namelist = "INCAR"
tag = "ENCUT"
database[package][namelist][tag]

# Generate QE database

In [ ]:
work_dir = 'temp'
def_files = ['PW/Doc/INPUT_PW.def', 'PP/Doc/INPUT_PROJWFC.def']
versions = ['6.7', '6.8', '7.0', '7.1', '7.2']
from importlib import resources
database_dir = resources.files("codex.database")

for v in versions:
#    run_helpdoc(v)
    generate_database(v)

# Generate Codex

In [ ]:
from importlib import resources
filename = 'example/sr3pbo_bands.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
codex = Codex(filename, version)


In [ ]:
print(codex.filename)
print(codex.code)
print(codex.package)
print(codex.tags['control'])

In [ ]:
from importlib import resources
import os
from base64 import b64decode
import json
import shutil

dbversion='7.2'
dir='static'

base_db_dir = resources.files("codex.database")
code = "qe"  # Need better terminology
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)

working_dir = os.path.abspath(dir)
tags_dir = os.path.join(working_dir, "tags-qe")
if os.path.exists(tags_dir):
    shutil.rmtree(tags_dir, ignore_errors=True)
os.mkdir(tags_dir)
for p, package in database.items():
    print(p)
    os.mkdir(os.path.join(tags_dir, p))
    for nl, namelist in package.items():
        print(nl)
        os.mkdir(os.path.join(tags_dir, p, nl))
        for tag, v in namelist.items():
            print(tag, v)
            if v["html"] != 'No documentation was found for this tag.':
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(tags_dir, p, nl, f"{tag}.html")
                with open(path, "w") as f:
                    f.write(webpage)

# CLI stuff

In [ ]:
from importlib.metadata import metadata
metadata = metadata('dft-codex')
__author__ = metadata['Author']
__version__ = metadata['Version']
__email__ = metadata['Author-email']
__maintainer__ = metadata['Maintainer']
__maintainer_email__ = metadata['Maintainer-email']
__summary__ = metadata['Summary']

print(list(metadata))
print(__author__)
print(__version__)
print(__email__)
print(__maintainer__)
print(__maintainer_email__)
print(__summary__)

from importlib.resources import files, contents
data_text = files('codex.database').joinpath('qe-7.2').joinpath('database.json')
# Check if it exists
data_text.exists()

In [ ]:
import webbrowser
webbrowser.get()

In [ ]:
import os


In [ ]:
import os
file = 'x/y/z/hi_bye.omak_sharmoota.html'
os.path.basename(file).split('.html')[0]

In [ ]:
import f90nml

def find_package(data, partial_match):
    for top_key, sub_dict in data.items():
        match = True
        for sub_key, sub_value in partial_match.items():
            if sub_key not in sub_dict or sub_dict[sub_key] != sub_value:
                match = False
                break
        if match:
            return top_key
    return None

# Load Database

input_filenames = ['example/pw.in', 'example/pw2.in']
input_data = [f90nml.read(f) for f in input_filenames]
find_package(database, input_data[0])

In [ ]:
data = {
    'hi': {
        'mike': {'a': 1, 'b': 2},
        'bob': {'c': 1, 'd': 2},
        'alice': {'e': 2, 'f': 30}
    },
    'bye': {
        'mike': {'x': 1, 'b': 2},
        'archie': {'c': 1, 'd': 2},
        'claudia': {'e': 2, 'f': 5, 'abc': 10}
    },
    'adios': {
        'marshall': {'x': 1, 'y': 2},
        'bob': {'c': 1, 'sdf': 234},
        'mikey': {'e': 2, 'f': 30}
    }
}

partial_match = {'mike': {'b': 2}, 'bob': {'c': 1}}
find_package(data, partial_match)

In [ ]:
import json
import f90nml
database_filename = 'codex/database/qe-7.2/database.json'
with open(database_filename) as f:
    database = json.load(f)

def mash(database):
    mashed = {}
    for package, namelists in database.items():
        mashed[package] = []
        for namelist, tags in namelists.items():
            for t in tags.keys():
                mashed[package].append(namelist + '.' + t)
    return mashed

input_filenames = ['example/sr3pbo_bands.in', 'example/si_bands.in', 'example/bi2te3_proj.in', 'example/bi4te3_bandsx.in']
input_data = [f90nml.read(f) for f in input_filenames]

flat_db = mash(database)
for i in range(len(input_data)):
    flat_input = mash({'?': input_data[i]})['?']
    print(flat_input)
    matches = []
    for package, mashed_tags in flat_db.items():
        if all([tag in mashed_tags for tag in flat_input]):
            matches.append(package)
    print(f'For input file {input_filenames[i]}, the following packages match: {matches}')

In [ ]:
print(flat_db['projwfc'])

In [ ]:

import json
import f90nml
database_filename = 'codex/database/qe-7.2/database.json'
with open(database_filename) as f:
    database = json.load(f)

In [ ]:
database['pw']['system']['lspinorb']

In [ ]:
database['bands']['bands']['lsigma']